In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import random

In [29]:
import glob
import os

In [30]:
def samples_num_in_window(frequency, window_size_ms):  
    return int(window_size_ms * frequency / 1000)

In [31]:
def read_emg(data_path):
    sessions_csv = []
    for path, _, files in os.walk(data_path):
        for name in files:
            sessions_csv.append(os.path.join(path, name))

    data = pd.concat([pd.read_csv(file, header = None) for file in sessions_csv]).values
    # print('input shape', data.shape)
    
    # reshape data
    # one column - one channel
    data_x = data[:,:-1]
    data_y = data[:,-1]
    data_x = data_x.reshape((-1, 8))
    data_y = data_y.repeat(8)
    data_y = data_y.reshape((-1,1))
    data = np.concatenate((data_x, data_y), axis=1)
    # print('result shape: ', data.shape)

    return data

In [32]:
def emg_data_windowing(data, window_size):
    data_win = np.copy(data)
    data_x = data_win[:,:-1]
    data_y = data_win[:,-1]
    n, m = data_x.shape
    size = n * m
    residual_rows_num =  n % window_size
    if residual_rows_num != 0:
        data_x = data_x[:-residual_rows_num,:]
        data_y = data_y[:-residual_rows_num]
    data_x = data_x.reshape((-1, m * window_size))
    
    data_y = data_y.reshape((-1, window_size))
    data_y = np.array(list(map(np.mean, data_y)))
    
    mixed_classes_idxs = np.where(data_y % 1 != 0)
    
    data_win = np.c_[data_x, data_y]
    data_win = np.delete(data_win, mixed_classes_idxs, 0)
    
    return data_win

In [33]:
import math

def integrated_absolute_value(segment):
    return sum([abs(s) for s in segment])

def mean_absolute_value(segment):
    return sum([abs(s) for s in segment])/len(segment)

def waveform_length(segment):
    n = len(segment)
    wl = 0
    for i in range(1, n):
        wl += abs(segment[i] - segment[i-1])
    return wl

def zero_crossing(segment):
    n = len(segment)
    zc = 0
    for i in range(n - 1):
        if segment[i] * segment[i+1] < 0:
            zc += 1
    return zc

def slope_sign_changes(segment):
    n = len(segment)
    ssc = 0
    for i in range(1, n-1):
        if segment[i-1] < segment[i] and segment[i] > segment[i+1] or segment[i-1] > segment[i] and segment[i] < segment[i+1]:
            ssc += 1
    return ssc

def root_mean_square(segment):
    return math.sqrt(sum([s*s for s in segment])/len(segment))

In [34]:
from nitime.algorithms.autoregressive import AR_est_LD
from sklearn.preprocessing import StandardScaler

def autoregression_coefficients(emg, order):
    coef = AR_est_LD(emg, order=order)[0]
    return coef

In [35]:
def calculate_features(data_x, channels_num, ar_features=True):
    n, m = data_x.shape
    features = []
    
    for channel in range(channels_num):
        channel_features = []
        
        # Calculate MAV, ZC, SSC, WL features
        channel_features.append(list(map(mean_absolute_value, data_x[:,channel::channels_num])))
        channel_features.append(list(map(waveform_length, data_x[:,channel::channels_num])))
        channel_features.append(list(map(zero_crossing, data_x[:,channel::channels_num])))
        channel_features.append(list(map(slope_sign_changes, data_x[:,channel::channels_num])))
        
        if ar_features:
            # calculate AR6 coefficients
            ar_order = 6
            ar_coef = np.array(list(map(lambda x: autoregression_coefficients(x, ar_order), data_x[:,channel::channels_num])))
            channel_features += ar_coef.transpose().tolist()
        
        features += channel_features
    
    return np.array(features).transpose()

In [41]:
def gesture_classification(train, test, input_frequency, window_size_ms, classifier, ar_features=True):
    window_samples = samples_num_in_window(input_frequency, window_size_ms)
    train_win = emg_data_windowing(train, window_samples)
    test_win = emg_data_windowing(test, window_samples)
    
    train_x = train_win[:,:-1]
    train_y = train_win[:,-1].astype('int')
    test_x = test_win[:,:-1]
    test_y = test_win[:,-1].astype('int') 
    
    train_features = calculate_features(train_x, 8, ar_features)
    test_features = calculate_features(test_x, 8, ar_features)
    
    classifier.fit(train_features, train_y)
    pred = classifier.predict(test_features)
    
    return metrics.accuracy_score(test_y, pred)

In [42]:
person_id = ['p001', 'p002', 'p003']

for p in person_id:
    # prepare data
    random.seed(101)
    print('person ' + p)
    session_1 = read_emg('data/gestures-9/' + p + '/session1')
    session_2 = read_emg('data/gestures-9/' + p + '/session2')
    session_3 = read_emg('data/gestures-9/' + p + '/session3')
    
    data_train = np.concatenate((session_1, session_2))
    data_test = session_3

    clf = svm.SVC(kernel='linear')
    accuracy = gesture_classification(data_train, data_test, 200, 200, clf)
    print("Accuracy:", accuracy)
    print()

person p001


c:\users\ekm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in remainder
  app.launch_new_instance()


Accuracy: 0.7911111111111111

person p002


c:\users\ekm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in remainder
  app.launch_new_instance()


Accuracy: 0.7022222222222222

person p003


c:\users\ekm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in remainder
  app.launch_new_instance()


Accuracy: 0.6333333333333333



* p001: 0.79
* p002: 0.70
* p003: 0.63

In [43]:
# train on all sessions of p001
# test on all sessions of p002

# prepare data
random.seed(101)

p1_all_sessions = read_emg('data/gestures-9/p001')
data_train = p1_all_sessions

p2_all_sessions = read_emg('data/gestures-9/p002')
data_test = p2_all_sessions

clf = svm.SVC(kernel='linear')
accuracy = gesture_classification(data_train, data_test, 200, 200, clf)
print("Accuracy:", accuracy)
print()

c:\users\ekm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in remainder
  app.launch_new_instance()


Accuracy: 0.17867867867867868

